In [1]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 22.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=6c13ec851a1bb1c05c1cab649642f377cd71913ac1d580d02c0b5bd4036e113f
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
from pyspark.ml import pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark=SparkSession.builder.appName("classification").getOrCreate()

# Read The Data set in pyspark

In [8]:
data = spark.read.format("csv").load("/content/balance-scale.data",header = True)

In [9]:
data.show()

+---+---+---+---+---+
|  B| 11| 12| 13| 14|
+---+---+---+---+---+
|  R|  1|  1|  1|  2|
|  R|  1|  1|  1|  3|
|  R|  1|  1|  1|  4|
|  R|  1|  1|  1|  5|
|  R|  1|  1|  2|  1|
|  R|  1|  1|  2|  2|
|  R|  1|  1|  2|  3|
|  R|  1|  1|  2|  4|
|  R|  1|  1|  2|  5|
|  R|  1|  1|  3|  1|
|  R|  1|  1|  3|  2|
|  R|  1|  1|  3|  3|
|  R|  1|  1|  3|  4|
|  R|  1|  1|  3|  5|
|  R|  1|  1|  4|  1|
|  R|  1|  1|  4|  2|
|  R|  1|  1|  4|  3|
|  R|  1|  1|  4|  4|
|  R|  1|  1|  4|  5|
|  R|  1|  1|  5|  1|
+---+---+---+---+---+
only showing top 20 rows



In [10]:
data.printSchema()

root
 |-- B: string (nullable = true)
 |-- 11: string (nullable = true)
 |-- 12: string (nullable = true)
 |-- 13: string (nullable = true)
 |-- 14: string (nullable = true)



#change the data type in pyspark

In [12]:
data=data.withColumn('B',data['B'].astype('string'))\
.withColumn('11',data['11'].astype('float'))\
.withColumn('12',data['12'].astype('float'))\
.withColumn('13',data['13'].astype('float'))\
.withColumn('14',data['14'].astype('float'))

In [13]:
data.printSchema()

root
 |-- B: string (nullable = true)
 |-- 11: float (nullable = true)
 |-- 12: float (nullable = true)
 |-- 13: float (nullable = true)
 |-- 14: float (nullable = true)



In [15]:
 from pyspark.ml.feature import StringIndexer
stringindexer =  StringIndexer(
    inputCol = 'B',
    outputCol = 'B1'
)
     
 

In [16]:
data1 = stringindexer.fit(data)

In [20]:
data=data1.transform(data)

In [21]:
data.show()

+---+---+---+---+---+---+
|  B| 11| 12| 13| 14| B1|
+---+---+---+---+---+---+
|  R|1.0|1.0|1.0|2.0|1.0|
|  R|1.0|1.0|1.0|3.0|1.0|
|  R|1.0|1.0|1.0|4.0|1.0|
|  R|1.0|1.0|1.0|5.0|1.0|
|  R|1.0|1.0|2.0|1.0|1.0|
|  R|1.0|1.0|2.0|2.0|1.0|
|  R|1.0|1.0|2.0|3.0|1.0|
|  R|1.0|1.0|2.0|4.0|1.0|
|  R|1.0|1.0|2.0|5.0|1.0|
|  R|1.0|1.0|3.0|1.0|1.0|
|  R|1.0|1.0|3.0|2.0|1.0|
|  R|1.0|1.0|3.0|3.0|1.0|
|  R|1.0|1.0|3.0|4.0|1.0|
|  R|1.0|1.0|3.0|5.0|1.0|
|  R|1.0|1.0|4.0|1.0|1.0|
|  R|1.0|1.0|4.0|2.0|1.0|
|  R|1.0|1.0|4.0|3.0|1.0|
|  R|1.0|1.0|4.0|4.0|1.0|
|  R|1.0|1.0|4.0|5.0|1.0|
|  R|1.0|1.0|5.0|1.0|1.0|
+---+---+---+---+---+---+
only showing top 20 rows



# Using VectorAssembler to add feature column where all input columns combine as list 


# feature columns mai target column use nhi krte hai

In [23]:
from pyspark.ml.feature import VectorAssembler

In [24]:
va = VectorAssembler(
    inputCols=['11','12','13','14'],
    outputCol='features'
)

In [25]:
data = va.transform(data)

In [26]:
data.show()

+---+---+---+---+---+---+-----------------+
|  B| 11| 12| 13| 14| B1|         features|
+---+---+---+---+---+---+-----------------+
|  R|1.0|1.0|1.0|2.0|1.0|[1.0,1.0,1.0,2.0]|
|  R|1.0|1.0|1.0|3.0|1.0|[1.0,1.0,1.0,3.0]|
|  R|1.0|1.0|1.0|4.0|1.0|[1.0,1.0,1.0,4.0]|
|  R|1.0|1.0|1.0|5.0|1.0|[1.0,1.0,1.0,5.0]|
|  R|1.0|1.0|2.0|1.0|1.0|[1.0,1.0,2.0,1.0]|
|  R|1.0|1.0|2.0|2.0|1.0|[1.0,1.0,2.0,2.0]|
|  R|1.0|1.0|2.0|3.0|1.0|[1.0,1.0,2.0,3.0]|
|  R|1.0|1.0|2.0|4.0|1.0|[1.0,1.0,2.0,4.0]|
|  R|1.0|1.0|2.0|5.0|1.0|[1.0,1.0,2.0,5.0]|
|  R|1.0|1.0|3.0|1.0|1.0|[1.0,1.0,3.0,1.0]|
|  R|1.0|1.0|3.0|2.0|1.0|[1.0,1.0,3.0,2.0]|
|  R|1.0|1.0|3.0|3.0|1.0|[1.0,1.0,3.0,3.0]|
|  R|1.0|1.0|3.0|4.0|1.0|[1.0,1.0,3.0,4.0]|
|  R|1.0|1.0|3.0|5.0|1.0|[1.0,1.0,3.0,5.0]|
|  R|1.0|1.0|4.0|1.0|1.0|[1.0,1.0,4.0,1.0]|
|  R|1.0|1.0|4.0|2.0|1.0|[1.0,1.0,4.0,2.0]|
|  R|1.0|1.0|4.0|3.0|1.0|[1.0,1.0,4.0,3.0]|
|  R|1.0|1.0|4.0|4.0|1.0|[1.0,1.0,4.0,4.0]|
|  R|1.0|1.0|4.0|5.0|1.0|[1.0,1.0,4.0,5.0]|
|  R|1.0|1.0|5.0|1.0|1.0|[1.0,1.

#Split the train and test in pyspark

In [27]:
#Split
trainingData,testData =data.randomSplit([0.7,0.3])

In [28]:
trainingData.show()

+---+---+---+---+---+---+-----------------+
|  B| 11| 12| 13| 14| B1|         features|
+---+---+---+---+---+---+-----------------+
|  B|1.0|2.0|2.0|1.0|2.0|[1.0,2.0,2.0,1.0]|
|  B|1.0|4.0|1.0|4.0|2.0|[1.0,4.0,1.0,4.0]|
|  B|2.0|1.0|1.0|2.0|2.0|[2.0,1.0,1.0,2.0]|
|  B|2.0|1.0|2.0|1.0|2.0|[2.0,1.0,2.0,1.0]|
|  B|2.0|2.0|1.0|4.0|2.0|[2.0,2.0,1.0,4.0]|
|  B|2.0|3.0|2.0|3.0|2.0|[2.0,3.0,2.0,3.0]|
|  B|2.0|4.0|2.0|4.0|2.0|[2.0,4.0,2.0,4.0]|
|  B|2.0|4.0|4.0|2.0|2.0|[2.0,4.0,4.0,2.0]|
|  B|2.0|5.0|2.0|5.0|2.0|[2.0,5.0,2.0,5.0]|
|  B|3.0|1.0|1.0|3.0|2.0|[3.0,1.0,1.0,3.0]|
|  B|3.0|4.0|4.0|3.0|2.0|[3.0,4.0,4.0,3.0]|
|  B|3.0|5.0|5.0|3.0|2.0|[3.0,5.0,5.0,3.0]|
|  B|4.0|1.0|2.0|2.0|2.0|[4.0,1.0,2.0,2.0]|
|  B|4.0|1.0|4.0|1.0|2.0|[4.0,1.0,4.0,1.0]|
|  B|4.0|2.0|4.0|2.0|2.0|[4.0,2.0,4.0,2.0]|
|  B|4.0|3.0|4.0|3.0|2.0|[4.0,3.0,4.0,3.0]|
|  B|4.0|5.0|4.0|5.0|2.0|[4.0,5.0,4.0,5.0]|
|  B|5.0|1.0|1.0|5.0|2.0|[5.0,1.0,1.0,5.0]|
|  B|5.0|1.0|5.0|1.0|2.0|[5.0,1.0,5.0,1.0]|
|  B|5.0|3.0|3.0|5.0|2.0|[5.0,3.

In [29]:
testData.show()

+---+---+---+---+---+---+-----------------+
|  B| 11| 12| 13| 14| B1|         features|
+---+---+---+---+---+---+-----------------+
|  B|1.0|2.0|1.0|2.0|2.0|[1.0,2.0,1.0,2.0]|
|  B|1.0|3.0|1.0|3.0|2.0|[1.0,3.0,1.0,3.0]|
|  B|1.0|3.0|3.0|1.0|2.0|[1.0,3.0,3.0,1.0]|
|  B|1.0|4.0|2.0|2.0|2.0|[1.0,4.0,2.0,2.0]|
|  B|1.0|4.0|4.0|1.0|2.0|[1.0,4.0,4.0,1.0]|
|  B|1.0|5.0|1.0|5.0|2.0|[1.0,5.0,1.0,5.0]|
|  B|1.0|5.0|5.0|1.0|2.0|[1.0,5.0,5.0,1.0]|
|  B|2.0|2.0|2.0|2.0|2.0|[2.0,2.0,2.0,2.0]|
|  B|2.0|2.0|4.0|1.0|2.0|[2.0,2.0,4.0,1.0]|
|  B|2.0|3.0|3.0|2.0|2.0|[2.0,3.0,3.0,2.0]|
|  B|2.0|5.0|5.0|2.0|2.0|[2.0,5.0,5.0,2.0]|
|  B|3.0|1.0|3.0|1.0|2.0|[3.0,1.0,3.0,1.0]|
|  B|3.0|2.0|2.0|3.0|2.0|[3.0,2.0,2.0,3.0]|
|  B|3.0|2.0|3.0|2.0|2.0|[3.0,2.0,3.0,2.0]|
|  B|3.0|3.0|3.0|3.0|2.0|[3.0,3.0,3.0,3.0]|
|  B|3.0|4.0|3.0|4.0|2.0|[3.0,4.0,3.0,4.0]|
|  B|3.0|5.0|3.0|5.0|2.0|[3.0,5.0,3.0,5.0]|
|  B|4.0|1.0|1.0|4.0|2.0|[4.0,1.0,1.0,4.0]|
|  B|4.0|2.0|2.0|4.0|2.0|[4.0,2.0,2.0,4.0]|
|  B|4.0|3.0|3.0|4.0|2.0|[4.0,3.

#Create model labelCol = Target col, featureCol= features

In [30]:
dtmodel = DecisionTreeClassifier(labelCol="B1",featuresCol="features")

In [31]:
dtmodelfit = dtmodel.fit(trainingData)

In [32]:
predictions = dtmodelfit.transform(testData)

In [33]:
predictions.columns

['B',
 '11',
 '12',
 '13',
 '14',
 'B1',
 'features',
 'rawPrediction',
 'probability',
 'prediction']

# select the prediction and infected column from pridictions

In [34]:
predictions.select("prediction","B1").show()

+----------+---+
|prediction| B1|
+----------+---+
|       1.0|2.0|
|       1.0|2.0|
|       1.0|2.0|
|       0.0|2.0|
|       1.0|2.0|
|       0.0|2.0|
|       1.0|2.0|
|       0.0|2.0|
|       0.0|2.0|
|       0.0|2.0|
|       0.0|2.0|
|       1.0|2.0|
|       1.0|2.0|
|       0.0|2.0|
|       0.0|2.0|
|       0.0|2.0|
|       0.0|2.0|
|       0.0|2.0|
|       1.0|2.0|
|       0.0|2.0|
+----------+---+
only showing top 20 rows



# Evalution

In [35]:

eval = MulticlassClassificationEvaluator(
    labelCol = "B1",
    predictionCol = "prediction",
    metricName = "accuracy"
)

In [36]:
accuracy = eval.evaluate(predictions)

In [37]:
print(accuracy)

0.7593582887700535
